In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import cobra
import cplex
import re
import os
from os.path import exists
import logging
from configparser import ConfigParser
config = ConfigParser()
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
import cobrakbase
import cobra
from escher import Builder
from optlang.symbolics import Zero, add
from modelseedpy import MSPackageManager, MSGapfill, FBAHelper, MSGrowthPhenotypes, MSModelUtil, MSATPCorrection
from cobrakbase.core.kbasefba.newmodeltemplate_builder import NewModelTemplateBuilder
from annotation_ontology_api.annotation_ontology_apiServiceClient import annotation_ontology_api
from modelseedpy.helpers import get_template
from sklearn.metrics import r2_score
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML
print("Required modules loaded")

python version 3.7.6
cobrakbase 0.2.8
Required modules loaded


In [6]:
#First we pull all the Biolog datasets out of Kbase
wsid = 114100
kbase_api = cobrakbase.KBaseAPI()
biologs = kbase_api.list_objects(wsid, object_type="KBasePhenotypes.PhenotypeSet", include_metadata=True)
genome_biologs = {}
for item in biologs:
    if "GenomeID" in item[10]:
        kbpheno = kbase_api.get_object(item[1], wsid)
        genome_biologs[item[10]["GenomeID"]] = MSGrowthPhenotypes.from_kbase_object(kbpheno,kbase_api)
#Now we iterate over the biolog datasets, pull the model, and simulate the phenotypes
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
pheno_frequency = {}
pheno_reactions = {}
completed_genomes = {}
#NOTE - this loop may timeout or crash - just rerun this code cell until it completes

In [ ]:
#Now we simulate every Biolog with the models and save the active reactions
#NOTE - again, this loop may timeout or crash - just rerun THIS codecell repeatedly until it finishes - progress is saved from each run
for genome in genome_biologs:
    if genome not in completed_genomes:
        model = kbase_api.get_from_ws(genome+".fbamodel",wsid)
        for pheno in genome_biologs[genome].phenotypes:
            if pheno.id not in pheno_frequency:
                pheno_frequency[pheno.id] = 0
                pheno_reactions[pheno.id] = {}
            pheno_frequency[pheno.id] += 1
            if pheno.growth > 0:
                output = pheno.simulate(model,add_missing_exchanges=True,save_fluxes=True,pfba=True)
                if output["growth"] < 0.00001:
                    pheno.gapfill_model_for_phenotype(model,[template],None,add_missing_exchanges=True)
                    output = pheno.simulate(model,add_missing_exchanges=True,save_fluxes=True,pfba=True)
                if output["growth"] > 0.01:
                    for rxn, value in output["fluxes"].items():
                        if abs(value) > 0:
                            if rxn not in pheno_reactions[pheno.id]:
                                pheno_reactions[pheno.id][rxn] = 0
                            pheno_reactions[pheno.id][rxn] += 1
        df = pd.DataFrame(pheno_reactions)
        df.to_csv("BiologFeatureAnalysis.csv")
        completed_genomes[genome] = 1